In [11]:
from projects.NeuralForceField.train import *
import numpy as np

from sklearn.utils import shuffle
from projects.graphbuilder.graphbuilder import Graph, GraphDataset

In [45]:
# ethanol data 
train = np.load('/home/wwj/data/ethanol_ccsd_t-train.npz')
xyz_data = np.dstack((np.array([train.f.z]*1000).reshape(1000, -1, 1), np.array(train.f.R)))
force_data = train.f.F
energy_data = train.f.E.squeeze() - train.f.E.mean()

In [6]:
# build graph dataset

# shuffle data
xyz_data, force_data, energy_data = shuffle(xyz_data, force_data, energy_data)
mean = np.array(energy_data).mean()
std = np.array(energy_data).std()

Fr = 4 # let node features be 
Fe = 1
dynamic_adj_mat = True
cutoff = 5.0
batch_size = 100
graph_data = GraphDataset(dynamic_adj_mat=dynamic_adj_mat)
for index in range(len(energy_data)):
    
    xyz = np.array(xyz_data[index])
    force = np.array(force_data[index])
    energy = np.array(np.array(energy_data[index]))
    node = xyz[:, 0].reshape(-1, 1)
    graph = Graph(N=node.shape[0], dynamic_adj_mat=dynamic_adj_mat)
    node_force = np.hstack((node, force)) # node concatenate with force 
    graph.SetNodeLabels(r=torch.Tensor(node_force))
    graph.SetXYZ(xyz=torch.Tensor(xyz[:, 1:4]))
    graph.UpdateConnectivity(cutoff=cutoff)
    graph.SetEdgeLabels()
    graph.LabelEdgesWithDistances()
    graph.SetGraphLabel(torch.Tensor(energy))
    graph_data.AddGraph(graph)
    
# batching graph 
graph_data.CreateBatches(batch_size=batch_size)

In [7]:
# initialize parameters
par = dict()

par["n_filters"] = 256
par["n_gaussians"] = 32
par["n_atom_basis"] = 256
par["optim"] = 1e-4
par["scheduler"] = True
par["train_percentage"] = 0.8
par["T"] = 10
par["batch_size"] = 100
par["cutoff"] = 5.0 
par["max_epoch"] = 5000
par["trainable_gauss"] = True

In [8]:
model = Model(par=par, graph_data=graph_data, device=3, job_name="test", graph_batching=True, root='./log/')

In [9]:
model.train(10)

/home/wwj/Repo/projects/NeuralForceField/train.py:127: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  train_u = train_u_mae.data[0]/self.N_train
/home/wwj/Repo/projects/NeuralForceField/train.py:128: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  train_force = train_force_mae.data[0]/self.N_train


epoch 0  U train: 4.789  force train 18.172
epoch 1  U train: 2.690  force train 13.784
epoch 2  U train: 2.524  force train 10.468
epoch 3  U train: 2.690  force train 8.697
epoch 4  U train: 2.112  force train 7.180
epoch 5  U train: 2.582  force train 6.347
epoch 6  U train: 2.708  force train 5.743
epoch 7  U train: 2.689  force train 5.275
epoch 8  U train: 2.732  force train 4.909
epoch 9  U train: 2.722  force train 4.611
